In [11]:
import pandas as pd
import gzip
import os
import matplotlib.pyplot as plt

TCGA_path = './TCGA/'
Xenium_path = './XeniumPanels/'

In [12]:
TCGA_dir = os.listdir(TCGA_path)
TCGA_files = [file for file in TCGA_dir if file.endswith('.tsv')]

Xenium_dir = os.listdir(Xenium_path)
Xenium_files = [file for file in Xenium_dir if file.endswith('.csv')]

In [13]:
print(Xenium_files)
Xenium_titles = Xenium_files.copy()
for i in range(len(Xenium_titles)):
    Xenium_titles[i] = Xenium_titles[i].split('_')[1]
print(Xenium_titles)

['Xenium_hSkin_preview_metadata.csv', 'Xenium_hBrain_v1_metadata.csv', 'Xenium_hColon_preview_metadata.csv', 'Xenium_hBreast_v1_metadata.csv', 'Xenium_hLung_v1_metadata.csv', 'Xenium_hMulti_v1_metadata.csv']
['hSkin', 'hBrain', 'hColon', 'hBreast', 'hLung', 'hMulti']


In [14]:
TCGA_tissue_type = ['Esophageal Carcinoma', 'Sarcoma', 'Testis Germ Cell Neoplasms', 'Kidney Adenomas and Adenocarcinomas', 'Skin Nevi and Melanomas', 
    'Phenochromocytoma and Paraganglioma', 'Brain Gliomas', 'Head and Neck Squamous Cell Carcinoma', 'Acute Myeloid Leukemia', 'Rhabdoid Tumor', 'Colon Adenocarcinoma', 
    'Liver and Intrahepatic Bile Ducts Adenomas and Adenocarcinomas', 'Pancreas', 'Uterine Corpus Endometrial Carcinoma', 'Cervix Uteri', 
    'Hematopoietic and Reticuloendothelial Systems Myeloid Leukemias', 'Kidney Complex Mixed and Stromal Neoplasms', 'Rectum Adenocarcinoma', 
    'Adrenal Gland Adenomas and Adenocarcinomas', 'Prostate Gland', 'Kidney Adenomas and Adenocarcinomas', 'Uterus, NOS Complex Mixed and Stromal Neoplasms', 'Thyroid Gland',
    'Breast', 'Ovary', 'Stomach', 'Bronchus and Lung Squamous Cell Neoplasms', 'Brain', 'Thymoma', 'Bladder', 'Mesothelioma', 'Kidney Adenomas and Adenocarcinomas',
    'Eyes and Adenexa Nevi and Melanomas', 'Lymphoid Neoplasm Diffuse Large B-cell Lymphoma', 'Cholangiocarcinoma', 'Bronchus and Lung']

In [15]:
for x in range(len(Xenium_files)):
    
    gene_list_path = Xenium_path+Xenium_files[x]
    df_genes = pd.read_csv(gene_list_path)

    TCGA_names = TCGA_files.copy()
    for i in range(len(TCGA_names)):
        TCGA_names[i] = TCGA_names[i].replace('.tsv', '')

    df_csv = df_genes[['Ensembl_ID', 'Gene']]
    df_csv.set_index('Ensembl_ID', inplace=True)
    for name in TCGA_tissue_type:
        df_csv[name] = 'MISSING'
            
    for t in range(len(TCGA_files)):

        df = pd.read_csv(TCGA_path+TCGA_files[t], sep='\t', skiprows=[0])
        df = df[df['gene_id'].str.startswith('ENSG')]
        df.reset_index(drop=True, inplace=True)

        ensemble_ids = df_genes['Ensembl_ID'].unique()
        df['gene_id'] = df['gene_id'].str.split('.').str[0]
        filtered_df = df[df['gene_id'].isin(ensemble_ids)]
        filtered_df.set_index('gene_id', inplace=True)
        filtered_df = filtered_df.groupby('gene_id').sum()
        
        averages = filtered_df['tpm_unstranded']

        for index, row in df_csv.iterrows():
            if index in averages.index:
                # if averages[index] > 1:
                #     df_csv.at[index, TCGA_tissue_type[t]] = True
                # else:
                #     df_csv.at[index, TCGA_tissue_type[t]] = False
                df_csv.at[index, TCGA_tissue_type[t]] = averages[index]
    
    df_csv.to_csv('./TCGA_'+Xenium_titles[x]+'_count.csv', index=True)


/tmp/ipykernel_2950744/3481273686.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv[name] = 'MISSING'
/tmp/ipykernel_2950744/3481273686.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv[name] = 'MISSING'
/tmp/ipykernel_2950744/3481273686.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde